<a href="https://colab.research.google.com/github/DexterAct/DexterAct/blob/main/Project_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

file_path = '/content/190623_rumahcom_tangsel_city_unfiltered.csv'
df = pd.read_csv(file_path, encoding='latin1')

### Data Overview
Let's take a look at the first few rows of the DataFrame to understand its structure and content.

In [5]:
display(df.head())

,nav-link href,listing-location,price,bed,bath,listing-floorarea,listing-floorarea 2
0,https://www.rumah.com/listing-properti/dijual-...,"Gading Serpong, Tangerang Selatan, Banten","6,9 M",5.0,5.0,420 m²,Rp 20.720.721 per m²
1,https://www.rumah.com/listing-properti/dijual-...,"Gading Serpong, Tangerang Selatan, Banten","4,5 M",3.0,3.0,190 m²,Rp 12.747.875 per m²
2,https://www.rumah.com/listing-properti/dijual-...,"Gading Serpong, Tangerang Selatan, Banten","3,95 M",3.0,3.0,132 m²,Rp 30.859.375 per m²
3,https://www.rumah.com/listing-properti/dijual-...,"Gading Serpong, Tangerang Selatan, Banten","3,3 M",4.0,3.0,220 m²,Rp 18.333.333 per m²
4,https://www.rumah.com/listing-properti/dijual-...,"Gading Serpong, Tangerang Selatan, Banten","3,5 M",3.0,2.0,180 m²,Rp 27.777.778 per m²


### DataFrame Information
Now, let's check the data types and non-null values for each column to identify potential issues like missing data or incorrect data types.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29420 entries, 0 to 29419
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   nav-link href        29420 non-null  object 
 1   listing-location     29420 non-null  object 
 2   price                29420 non-null  object 
 3   bed                  29282 non-null  float64
 4   bath                 29215 non-null  float64
 5   listing-floorarea    29420 non-null  object 
 6   listing-floorarea 2  29383 non-null  object 
dtypes: float64(2), object(5)
memory usage: 1.6+ MB


### Descriptive Statistics
Finally, let's look at the descriptive statistics of the numerical columns to get a sense of their distribution and potential outliers.

In [7]:
display(df.describe(include='all'))

,nav-link href,listing-location,price,bed,bath,listing-floorarea,listing-floorarea 2
count,29420,29420,29420,29282.000000,29215.000000,29420,29383
unique,24179,8495,2116,NaN,NaN,533,5891
top,https://www.rumah.com/listing-properti/dijual-...,"Bintaro, Tangerang Selatan, Banten","3,5 M",NaN,NaN,200 m²,Rp 20.000.000 per m²
freq,5,4464,631,NaN,NaN,1349,417
mean,NaN,NaN,NaN,3.754457,2.986137,NaN,NaN
std,NaN,NaN,NaN,1.356762,1.359788,NaN,NaN
min,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN
25%,NaN,NaN,NaN,3.000000,2.000000,NaN,NaN
50%,NaN,NaN,NaN,4.000000,3.000000,NaN,NaN
75%,NaN,NaN,NaN,4.000000,4.000000,NaN,NaN


### Data Cleaning: Price Column

I'll start by cleaning the `price` column. It's currently stored as text with currency symbols and text representations of large numbers (e.g., 'Miliar' for billion, 'Juta' for million). The following code will convert these into a consistent numerical format (float).

In [26]:
import numpy as np

def clean_price(price_str):
    if isinstance(price_str, str):
        # Replace non-breaking spaces, 'Rp', and strip
        price_str = price_str.replace('Rp\u00a0', '').replace('Rp', '').strip()

        multiplier = 1
        # Check for 'Miliar', 'Juta', 'jt', 'M', 'rb'
        if 'Miliar' in price_str:
            multiplier = 1_000_000_000
            price_str = price_str.replace('\u00a0Miliar', '').replace(' Miliar', '')
        elif 'Juta' in price_str or 'jt' in price_str:
            multiplier = 1_000_000
            price_str = price_str.replace('\u00a0Juta', '').replace(' Juta', '')
            price_str = price_str.replace('\u00a0jt', '').replace(' jt', '')
        elif ' M' in price_str:
            multiplier = 1_000_000
            price_str = price_str.replace('\u00a0M', '').replace(' M', '')
        elif 'rb' in price_str:
            multiplier = 1_000
            price_str = price_str.replace('\u00a0rb', '').replace(' rb', '')

        # If it's a range, we'll mark it as NaN for now
        if '-' in price_str:
            return np.nan

        # Now, clean the numeric part: remove thousands separators ('.')
        # and convert decimal comma (',') to a dot ('.')
        numeric_part = price_str.replace('.', '').replace(',', '.').strip()
        try:
            return float(numeric_part) * multiplier
        except ValueError:
            return np.nan # Return NaN for unparseable strings
    return np.nan # Return NaN for non-string types or missing values

df['price_cleaned'] = df['price'].apply(clean_price)

print("Original 'price' and new 'price_cleaned' columns:")
display(df[['price', 'price_cleaned']].head())

Original 'price' and new 'price_cleaned' columns:


,price,price_cleaned
0,"6,9 M",6900000.0
1,"4,5 M",4500000.0
2,"3,95 M",3950000.0
3,"3,3 M",3300000.0
4,"3,5 M",3500000.0


### Data Cleaning: Floor Area Columns

Next, I'll clean `listing-floorarea` and `listing-floorarea 2`. These columns contain ' m²' and are also of object type. I'll extract the numerical part and convert them to float.

In [22]:
def clean_floor_area(area_str):
    if isinstance(area_str, str):
        # Remove ' m²' and strip whitespace, convert comma to dot for decimals
        area_str = area_str.replace(' m²', '').replace(',', '.').strip()
        try:
            return float(area_str)
        except ValueError:
            return np.nan
    return np.nan # Return NaN for non-string types or missing values

def clean_price_per_sqm(price_per_sqm_str):
    if isinstance(price_per_sqm_str, str):
        # Replace non-breaking spaces, 'Rp', ' per m²', thousand separators (dots), and convert decimal comma to dot
        price_per_sqm_str = price_per_sqm_str.replace('Rp\u00a0', '').replace('Rp', '')
        price_per_sqm_str = price_per_sqm_str.replace('\u00a0per\u00a0m\u00b2', '').replace(' per m²', '')
        price_per_sqm_str = price_per_sqm_str.strip()

        # Now, clean the numeric part: remove thousands separators ('.')
        # and convert decimal comma (',') to a dot ('.')
        numeric_part = price_per_sqm_str.replace('.', '').replace(',', '.').strip()
        try:
            return float(numeric_part)
        except ValueError:
            return np.nan
    return np.nan # Return NaN for non-string types or missing values

df['listing-floorarea_cleaned'] = df['listing-floorarea'].apply(clean_floor_area)
# Apply the new cleaning function for price per sqm and rename the column
df['price_per_sqm_cleaned'] = df['listing-floorarea 2'].apply(clean_price_per_sqm)

print("Original and cleaned floor area and price per sqm columns:")
display(df[['listing-floorarea', 'listing-floorarea_cleaned', 'listing-floorarea 2', 'price_per_sqm_cleaned']].head())

Original and cleaned floor area and price per sqm columns:


,listing-floorarea,listing-floorarea_cleaned,listing-floorarea 2,price_per_sqm_cleaned
0,420 m²,420.0,Rp 20.720.721 per m²,20720721.0
1,190 m²,190.0,Rp 12.747.875 per m²,12747875.0
2,132 m²,132.0,Rp 30.859.375 per m²,30859375.0
3,220 m²,220.0,Rp 18.333.333 per m²,18333333.0
4,180 m²,180.0,Rp 27.777.778 per m²,27777778.0


### Handling Missing Values

Finally, I'll address missing values in `bed`, `bath`, and the newly cleaned `listing-floorarea_2_cleaned` columns. I'll impute them with their respective medians, as the median is a robust measure that is less affected by outliers.

In [23]:
# Columns to impute with median
# 'listing-floorarea_2_cleaned' is replaced with 'price_per_sqm_cleaned'
columns_to_impute = ['bed', 'bath', 'price_per_sqm_cleaned']

for col in columns_to_impute:
    if col in df.columns and df[col].isnull().any():
        median_val = df[col].median()
        # Address FutureWarning: assign the result back instead of using inplace=True
        df[col] = df[col].fillna(median_val)
        print(f"Missing values in '{col}' imputed with median: {median_val}")

# Verify that there are no more missing values in the relevant columns
print("\nChecking for missing values after imputation:")
print(df[columns_to_impute + ['price_cleaned', 'listing-floorarea_cleaned']].isnull().sum())

# Display the DataFrame information after all cleaning and imputation steps
print("\nDataFrame info after cleaning and imputation:")
df.info()

Missing values in 'price_per_sqm_cleaned' imputed with median: 18333333.0

Checking for missing values after imputation:
bed                           0
bath                          0
price_per_sqm_cleaned         0
price_cleaned                18
listing-floorarea_cleaned     0
dtype: int64

DataFrame info after cleaning and imputation:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29420 entries, 0 to 29419
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   nav-link href                29420 non-null  object 
 1   listing-location             29420 non-null  object 
 2   price                        29420 non-null  object 
 3   bed                          29420 non-null  float64
 4   bath                         29420 non-null  float64
 5   listing-floorarea            29420 non-null  object 
 6   listing-floorarea 2          29383 non-null  object 
 7   price_cleaned          

### Inspecting Uncleaned 'price' values
Let's look at some of the 'price' values that resulted in `NaN` after cleaning to understand the remaining issues.

### Inspecting Uncleaned 'price' values (re-check)
Let's look again at some of the original 'price' values that became `NaN` after cleaning, as some might still be unparsed.

In [24]:
print("Sample of 'price' values that became NaN after cleaning:")
display(df['price'][df['price_cleaned'].isnull()].value_counts().head(20))

Sample of 'price' values that became NaN after cleaning:


,count
price,
900 rb,3
1 rb,2
"2,2 M - Rp 2,7 M",1
800 jt - Rp 1 M,1
875 rb,1
"875,5 jt - Rp 1,1 M",1
"1,08 M - Rp 2,3 M",1
600 rb,1
300 rb,1


### Inspecting Uncleaned 'listing-floorarea 2' values (critical re-check)
Given that `price_per_sqm_cleaned` is entirely `NaN`, we need to critically examine the *original* `listing-floorarea 2` values. I will display the most frequent unique values from this column to understand the formats that are failing to parse.

In [25]:
print("Most common original values in 'listing-floorarea 2' (as price per sqm is entirely NaN):")
display(df['listing-floorarea 2'].value_counts().head(20))

Most common original values in 'listing-floorarea 2' (as price per sqm is entirely NaN):


,count
listing-floorarea 2,
Rp 20.000.000 per m²,417
Rp 16.666.667 per m²,348
Rp 10.000.000 per m²,342
Rp 25.000.000 per m²,329
Rp 15.000.000 per m²,251
Rp 20.833.333 per m²,214
Rp 18.750.000 per m²,208
Rp 18.333.333 per m²,198
Rp 12.500.000 per m²,196


In [17]:
print("Sample of 'price' values that became NaN after cleaning:")
display(df['price'][df['price_cleaned'].isnull()].value_counts().head(20))

Sample of 'price' values that became NaN after cleaning:


,count
price,
900 jt,367
600 jt,308
850 jt,235
800 jt,205
700 jt,189
650 jt,160
950 jt,137
750 jt,125
780 jt,84


### Inspecting Uncleaned 'listing-floorarea 2' values
Now, let's examine 'listing-floorarea 2' values (intended for price per sqm) that resulted in `NaN` to improve the `clean_price_per_sqm` function.

In [18]:
print("Sample of 'listing-floorarea 2' values that became NaN after cleaning:")
display(df['listing-floorarea 2'][df['price_per_sqm_cleaned'].isnull()].value_counts().head(20))

Sample of 'listing-floorarea 2' values that became NaN after cleaning:


,count
listing-floorarea 2,


### Inspecting Uncleaned 'price' values
Let's look at some of the original 'price' values that resulted in `NaN` after cleaning to understand the remaining issues.

In [19]:
print("Sample of 'price' values that became NaN after cleaning:")
display(df['price'][df['price_cleaned'].isnull()].value_counts().head(20))

Sample of 'price' values that became NaN after cleaning:


,count
price,
900 jt,367
600 jt,308
850 jt,235
800 jt,205
700 jt,189
650 jt,160
950 jt,137
750 jt,125
780 jt,84


### Inspecting Uncleaned 'listing-floorarea 2' values
Now, let's examine 'listing-floorarea 2' values (intended for price per sqm) that resulted in `NaN` to improve the `clean_price_per_sqm` function. This column seems to be entirely unparsed, so we need to pay close attention to its formats.

In [20]:
print("Sample of 'listing-floorarea 2' values that became NaN after cleaning:")
display(df['listing-floorarea 2'][df['price_per_sqm_cleaned'].isnull()].value_counts().head(20))

Sample of 'listing-floorarea 2' values that became NaN after cleaning:


,count
listing-floorarea 2,


In [27]:
df

,nav-link href,listing-location,price,bed,bath,listing-floorarea,listing-floorarea 2,price_cleaned,listing-floorarea_cleaned,listing-floorarea_2_cleaned,price_per_sqm_cleaned
0,https://www.rumah.com/listing-properti/dijual-...,"Gading Serpong, Tangerang Selatan, Banten","6,9 M",5.0,5.0,420 m²,Rp 20.720.721 per m²,6900000.0,420.0,NaN,20720721.0
1,https://www.rumah.com/listing-properti/dijual-...,"Gading Serpong, Tangerang Selatan, Banten","4,5 M",3.0,3.0,190 m²,Rp 12.747.875 per m²,4500000.0,190.0,NaN,12747875.0
2,https://www.rumah.com/listing-properti/dijual-...,"Gading Serpong, Tangerang Selatan, Banten","3,95 M",3.0,3.0,132 m²,Rp 30.859.375 per m²,3950000.0,132.0,NaN,30859375.0
3,https://www.rumah.com/listing-properti/dijual-...,"Gading Serpong, Tangerang Selatan, Banten","3,3 M",4.0,3.0,220 m²,Rp 18.333.333 per m²,3300000.0,220.0,NaN,18333333.0
4,https://www.rumah.com/listing-properti/dijual-...,"Gading Serpong, Tangerang Selatan, Banten","3,5 M",3.0,2.0,180 m²,Rp 27.777.778 per m²,3500000.0,180.0,NaN,27777778.0
...,...,...,...,...,...,...,...,...,...,...,...
29415,https://www.rumah.com/listing-properti/dijual-...,"Bintaro, Tangerang Selatan, Banten","3,3 M",3.0,3.0,300 m²,Rp 21.710.526 per m²,3300000.0,300.0,NaN,21710526.0
29416,https://www.rumah.com/listing-properti/dijual-...,"Bintaro, Tangerang Selatan, Banten",4 M,8.0,4.0,330 m²,Rp 36.363.636 per m²,4000000.0,330.0,NaN,36363636.0
29417,https://www.rumah.com/listing-properti/dijual-...,"Bintaro, Tangerang Selatan, Banten","2,2 M",3.0,2.0,160 m²,Rp 18.333.333 per m²,2200000.0,160.0,NaN,18333333.0
29418,https://www.rumah.com/listing-properti/dijual-...,"Bintaro, Tangerang Selatan, Banten","1,95 M",3.0,2.0,120 m²,Rp 21.666.667 per m²,1950000.0,120.0,NaN,21666667.0


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


from matplotlib import pyplot as plt
df['bed'].plot(kind='hist', bins=20, title='bed')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['bath'].plot(kind='hist', bins=20, title='bath')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['price_cleaned'].plot(kind='hist', bins=20, title='price_cleaned')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
df.plot(kind='scatter', x='listing-floorarea_cleaned', y='price_per_sqm_cleaned', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['bed'].plot(kind='line', figsize=(8, 4), title='bed')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['bath'].plot(kind='line', figsize=(8, 4), title='bath')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['price_cleaned'].plot(kind='line', figsize=(8, 4), title='price_cleaned')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['listing-floorarea_cleaned'].plot(kind='line', figsize=(8, 4), title='listing-floorarea_cleaned')
plt.gca().spines[['top', 'right']].set_visible(False)